<a href="https://colab.research.google.com/github/Renadoo1/teaching/blob/main/intro_nlp_practical_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intro to NLP Practical<br>
======================<br>
Students will work through problems on n-grams, probabilities, OOV handling, and classifiers.<br>

In [ ]:
!python3 -m pip install nltk
!python3 -m pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.5 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 2/3 [nltk]]  WARNING: The script nltk is installed in '/Library/Frameworks/Python.framework/Versions/3.13/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [nltk]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 12.5 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 12.7 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

Toy corpus for language modeling

In [ ]:
corpus = [
    "Mary had a little lamb",
    "Its fleece was white as snow",
    "And everywhere that Mary went",
    "The lamb was sure to go"
]

--- Part 1: Preprocessing ---

 Q1.1 Sequence notation<br>
Exercise: Write sequence notation for the sentence:<br>
"Mary had a little lamb, its fleece was white as snow"

In [ ]:


sentence = "Mary had a little lamb, its fleece was white as snow"

# Remove commas, split into words
words = sentence.replace(',', '').split()

# Sequence notation
sequence_notation = [f"x_{i+1}={word}" for i, word in enumerate(words)]
print("Sequence Notation:")
print(sequence_notation)

# Print the number-indexed words
print("\nIndexed words:")
for i, word in enumerate(words, start=1):
    print(f"index {i}: {word}")


Sequence Notation:
['x_1=Mary', 'x_2=had', 'x_3=a', 'x_4=little', 'x_5=lamb', 'x_6=its', 'x_7=fleece', 'x_8=was', 'x_9=white', 'x_10=as', 'x_11=snow']

Indexed words:
index 1: Mary
index 2: had
index 3: a
index 4: little
index 5: lamb
index 6: its
index 7: fleece
index 8: was
index 9: white
index 10: as
index 11: snow


 Q1.2 Add start/end tokens<br>
Exercise: Write a function to tokenize the corpus and add <s>, </s>

In [ ]:

def add_start_end_tokens(corpus):
    tokenized_corpus = []
    for sentence in corpus:
        tokenized_sentence = ['<s>'] + sentence.split() + ['</s>']
        tokenized_corpus.append(tokenized_sentence)
    return tokenized_corpus

print("Tokenized Corpus with Start/End Tokens:")
print(tokenized_corpus)

Tokenized Corpus with Start/End Tokens:
[['<s>', 'Mary', 'had', 'a', 'little', 'lamb', '</s>'], ['<s>', 'Its', 'fleece', 'was', 'white', 'as', 'snow', '</s>'], ['<s>', 'And', 'everywhere', 'that', 'Mary', 'went', '</s>'], ['<s>', 'The', 'lamb', 'was', 'sure', 'to', 'go', '</s>']]


--- Part 2: N-grams & Probabilities ---

 Q2.1 Extract unigrams, bigrams, trigrams

In [ ]:

def extract_ngrams(tokenized_corpus, n):
    ngrams = []
    for sentence in tokenized_corpus:
        ngrams.extend(zip(*[sentence[i:] for i in range(n)]))
    return ngrams
unigrams = extract_ngrams(tokenized_corpus, 1)
bigrams = extract_ngrams(tokenized_corpus, 2)
trigrams = extract_ngrams(tokenized_corpus, 3)
print("\nUnigrams:")
print(unigrams)
print("\nBigrams:")
print(bigrams)
print("\nTrigrams:")
print(trigrams)


Unigrams:
[('<s>',), ('Mary',), ('had',), ('a',), ('little',), ('lamb',), ('</s>',), ('<s>',), ('Its',), ('fleece',), ('was',), ('white',), ('as',), ('snow',), ('</s>',), ('<s>',), ('And',), ('everywhere',), ('that',), ('Mary',), ('went',), ('</s>',), ('<s>',), ('The',), ('lamb',), ('was',), ('sure',), ('to',), ('go',), ('</s>',)]

Bigrams:
[('<s>', 'Mary'), ('Mary', 'had'), ('had', 'a'), ('a', 'little'), ('little', 'lamb'), ('lamb', '</s>'), ('<s>', 'Its'), ('Its', 'fleece'), ('fleece', 'was'), ('was', 'white'), ('white', 'as'), ('as', 'snow'), ('snow', '</s>'), ('<s>', 'And'), ('And', 'everywhere'), ('everywhere', 'that'), ('that', 'Mary'), ('Mary', 'went'), ('went', '</s>'), ('<s>', 'The'), ('The', 'lamb'), ('lamb', 'was'), ('was', 'sure'), ('sure', 'to'), ('to', 'go'), ('go', '</s>')]

Trigrams:
[('<s>', 'Mary', 'had'), ('Mary', 'had', 'a'), ('had', 'a', 'little'), ('a', 'little', 'lamb'), ('little', 'lamb', '</s>'), ('<s>', 'Its', 'fleece'), ('Its', 'fleece', 'was'), ('fleece', '

 Q2.2 Bigram probabilities<br>
Exercise: Write function to compute P(w_i | w_{i-1})

In [ ]:

def bigram_probabilities(bigrams):
    bigram_counts = Counter(bigrams)
    unigram_counts = Counter([bigram[0] for bigram in bigrams])
    probabilities = {}
    for (w1, w2), count in bigram_counts.items():
        probabilities[(w1, w2)] = count / unigram_counts[w1]
    return probabilities
print("\nBigram Probabilities:")
bigram_probs = bigram_probabilities(bigrams)
for (w1, w2), prob in bigram_probs.items():
    print(f"P({w2}|{w1}) = {prob:.4f}")



Bigram Probabilities:
P(Mary|<s>) = 0.2500
P(had|Mary) = 0.5000
P(a|had) = 1.0000
P(little|a) = 1.0000
P(lamb|little) = 1.0000
P(</s>|lamb) = 0.5000
P(Its|<s>) = 0.2500
P(fleece|Its) = 1.0000
P(was|fleece) = 1.0000
P(white|was) = 0.5000
P(as|white) = 1.0000
P(snow|as) = 1.0000
P(</s>|snow) = 1.0000
P(And|<s>) = 0.2500
P(everywhere|And) = 1.0000
P(that|everywhere) = 1.0000
P(Mary|that) = 1.0000
P(went|Mary) = 0.5000
P(</s>|went) = 1.0000
P(The|<s>) = 0.2500
P(lamb|The) = 1.0000
P(was|lamb) = 0.5000
P(sure|was) = 0.5000
P(to|sure) = 1.0000
P(go|to) = 1.0000
P(</s>|go) = 1.0000


 Q2.3 Sentence probability<br>
Exercise: Compute probability of "Mary had a little lamb"

In [ ]:

def sentence_probability(sentence, bigram_probs):
    words = ['<s>'] + sentence.replace(',', '').split() + ['</s>']
    prob = 1.0
    for i in range(1, len(words)):
        bigram = (words[i-1], words[i])
        prob *= bigram_probs.get(bigram, 0)
    return prob
sentence = "Mary had a little lamb"
prob = sentence_probability(sentence, bigram_probs)
print(f"\nProbability of the sentence '{sentence}': {prob:.10f}")


Probability of the sentence 'Mary had a little lamb': 0.0625000000


Q2.4 Handling OOV/UNK<br>
Exercise: Replace unseen words with <UNK> and recompute


In [ ]:

def replace_oov_with_unk(corpus, known_words):
    updated_corpus = []
    for sentence in corpus:
        updated_sentence = ' '.join([word if word in known_words else '<UNK>' for word in sentence.split()])
        updated_corpus.append(updated_sentence)
    return updated_corpus
known_words = set(word for sentence in tokenized_corpus for word in sentence)
updated_corpus = replace_oov_with_unk(corpus, known_words)
print("\nUpdated Corpus with <UNK>:")
print(updated_corpus)



Updated Corpus with <UNK>:
['Mary had a little lamb', 'Its fleece was white as snow', 'And everywhere that Mary went', 'The lamb was sure to go']


--- Part 3: Classifier ---

 Q3.1 Naive Bayes sentiment classifier

# 📽 Exercise 3.1: Sentiment Classification on toy dataset

In this exercise, you will build a simple sentiment classification model that predicts whether a given sentence is **positive** or **negative**.

---

## ✏️ Instructions:


### 1️⃣ Perform Feature Extraction
- Use **TF-IDF Vectorization** to convert names into numerical features.


---

### 2️⃣ Train a Machine Learning Classifier
- Use any classifier you are familiar with (e.g., **Logistic Regression** or **Naive Bayes**).
- Split the data into **training** and **testing** sets.
- Train the classifier on the training data.


🚀 **Goal:** By the end of this exercise, you should be able to:
- Apply **feature extraction** to text data.
- Train and evaluate a **text classification model** using **machine learning**.

In [ ]:
train_texts = [
    "I love my dog",
    "This food is great",
    "I hate waiting",
    "The movie was boring",
    "Happy with my phone",
    "This is awful"
]
train_labels = ["pos", "pos", "neg", "neg", "pos", "neg"]

# 📽 Exercise 3.2: Movie Review Classification using Movies Review Corpus

In this exercise, you will build a simple text classification model that predicts whether a given **movie review** is **positive** or **negative** using the **NLTK Movie Reviews Corpus**.

This is a classical example of text classification at the **sentence level**.

---

## ✏️ Instructions:

### 1️⃣ Load the Data
- Import the **Movie Reviews corpus** from **NLTK**.
- Create a dataset where each example is a review and the label is either `'positive'` or `'negative'`.

---

### 2️⃣ Perform Feature Extraction
- Use **TF-IDF Vectorization** to convert names into numerical features.


---

### 3️⃣ Train a Machine Learning Classifier
- Use any classifier you are familiar with (e.g., **Logistic Regression** or **Naive Bayes**).
- Split the data into **training** and **testing** sets.
- Train the classifier on the training data.

---

### 4️⃣ Evaluate the Classifier
- Use **accuracy** and a **classification report** to evaluate your model on the test set.
- Think about: How well does the model perform? Which reviews are harder to classify?

---

✅ You are free to explore:
- Trying different classifiers.
- Visualizing the results (e.g., confusion matrix).

---

🚀 **Goal:** By the end of this exercise, you should be able to:
- Apply **feature extraction** to text data.
- Train and evaluate a **text classification model** using **machine learning**.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/renadalshamrani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/renadalshamrani/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import nltk
import random
import string
from nltk.corpus import movie_reviews
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB

# 1️⃣ Load the Data
nltk.download('movie_reviews')
nltk.download('punkt')

documents = [(movie_reviews.raw(fileid), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

texts, labels = zip(*documents)
texts, labels = list(texts), list(labels)

# 2️⃣ Preprocess
def preprocess(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    return ' '.join(tokens)

texts = [preprocess(text) for text in texts]

# 3️⃣ Train/Test Split
X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# 4️⃣ Vectorize
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train_texts)
X_test = vectorizer.transform(X_test_texts)

# 5️⃣ Train Classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# 6️⃣ Evaluate
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))




[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/renadalshamrani/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/renadalshamrani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy: 0.7900

Classification Report:
              precision    recall  f1-score   support

         neg       0.78      0.81      0.80       202
         pos       0.80      0.77      0.78       198

    accuracy                           0.79       400
   macro avg       0.79      0.79      0.79       400
weighted avg       0.79      0.79      0.79       400



<Done By Best Group ever>

 Q3.3 Discussion: Why bigrams vs unigrams?<br>

--- Part 4: Wrap-up Reflection ---

 Q3.4 Limitations of n-grams

 Discussion Questions<br>
1. Why do we need <UNK> tokens?<br>
2. Why start/end tokens?<br>
3. Why not always use higher n-grams?<br>
4. How do classifiers differ from language models?